# Named Entity Recognition using sklearn-crfsuite

In this notebook we train a basic CRF model for Named Entity Recognition on CoNLL2002 data (following https://github.com/TeamHG-Memex/sklearn-crfsuite/blob/master/docs/CoNLL2002.ipynb) and check its weights to see what it learned.

To follow this tutorial you need NLTK > 3.x and sklearn-crfsuite Python packages. The tutorial uses Python 3.

In [1]:
import nltk
import sklearn_crfsuite
import eli5

## 1. Training data

CoNLL 2002 datasets contains a list of Spanish sentences, with Named Entities annotated. It uses [IOB2](https://en.wikipedia.org/wiki/Inside_Outside_Beginning) encoding. CoNLL 2002 data also provide POS tags.

In [2]:
train_sents = list(nltk.corpus.conll2002.iob_sents('esp.train'))
test_sents = list(nltk.corpus.conll2002.iob_sents('esp.testb'))
train_sents[0]

[('Melbourne', 'NP', 'B-LOC'),
 ('(', 'Fpa', 'O'),
 ('Australia', 'NP', 'B-LOC'),
 (')', 'Fpt', 'O'),
 (',', 'Fc', 'O'),
 ('25', 'Z', 'O'),
 ('may', 'NC', 'O'),
 ('(', 'Fpa', 'O'),
 ('EFE', 'NC', 'B-ORG'),
 (')', 'Fpt', 'O'),
 ('.', 'Fp', 'O')]

## 2. Feature extraction

POS tags can be seen as pre-extracted features. Let's extract more features (word parts, simplified POS tags, lower/title/upper flags, features of nearby words) and convert them to sklear-crfsuite format - each sentence should be converted to a list of dicts. This is a very simple baseline; you certainly can do better.

In [3]:
def word2features(sent, i):
    word = sent[i][0]
    postag = sent[i][1]
    
    features = {
        'bias': 1.0,
        'word.lower()': word.lower(),
        'word[-3:]': word[-3:],
        'word.isupper()': word.isupper(),
        'word.istitle()': word.istitle(),
        'word.isdigit()': word.isdigit(),
        'postag': postag,
        'postag[:2]': postag[:2],        
    }
    if i > 0:
        word1 = sent[i-1][0]
        postag1 = sent[i-1][1]
        features.update({
            '-1:word.lower()': word1.lower(),
            '-1:word.istitle()': word1.istitle(),
            '-1:word.isupper()': word1.isupper(),
            '-1:postag': postag1,
            '-1:postag[:2]': postag1[:2],
        })
    else:
        features['BOS'] = True
        
    if i < len(sent)-1:
        word1 = sent[i+1][0]
        postag1 = sent[i+1][1]
        features.update({
            '+1:word.lower()': word1.lower(),
            '+1:word.istitle()': word1.istitle(),
            '+1:word.isupper()': word1.isupper(),
            '+1:postag': postag1,
            '+1:postag[:2]': postag1[:2],
        })
    else:
        features['EOS'] = True
                
    return features


def sent2features(sent):
    return [word2features(sent, i) for i in range(len(sent))]

def sent2labels(sent):
    return [label for token, postag, label in sent]

def sent2tokens(sent):
    return [token for token, postag, label in sent]

X_train = [sent2features(s) for s in train_sents]
y_train = [sent2labels(s) for s in train_sents]

X_test = [sent2features(s) for s in test_sents]
y_test = [sent2labels(s) for s in test_sents]

This is how features extracted from a single token look like:

In [4]:
X_train[0][1]

{'+1:postag': 'NP',
 '+1:postag[:2]': 'NP',
 '+1:word.istitle()': True,
 '+1:word.isupper()': False,
 '+1:word.lower()': 'australia',
 '-1:postag': 'NP',
 '-1:postag[:2]': 'NP',
 '-1:word.istitle()': True,
 '-1:word.isupper()': False,
 '-1:word.lower()': 'melbourne',
 'bias': 1.0,
 'postag': 'Fpa',
 'postag[:2]': 'Fp',
 'word.isdigit()': False,
 'word.istitle()': False,
 'word.isupper()': False,
 'word.lower()': '(',
 'word[-3:]': '('}

## 3. Train a CRF model

Once we have features in a right format we can train a linear-chain CRF (Conditional Random Fields) model using sklearn_crfsuite.CRF:

In [5]:
crf = sklearn_crfsuite.CRF(
    algorithm='lbfgs',
    c1=0.1, 
    c2=0.1, 
    max_iterations=20,
    all_possible_transitions=False,
)
crf.fit(X_train, y_train);

## 4. Inspect model weights

CRFsuite CRF models use two kinds of features: state features and transition features. Let's check their weights 
using eli5.explain_weights:

In [6]:
eli5.show_weights(crf, top=30)

Transition features make sense: at least model learned that I-ENITITY must follow B-ENTITY. It also learned that some transitions are unlikely, e.g. it is not common in this dataset to have a location right after an organization name (I-ORG -> B-LOC has a large negative weight).

Features don't use gazetteers, so model had to remember some geographic names from the training data, e.g. that España is a location. 

If we regularize CRF more, we can expect that only features which are generic will remain, and memoized tokens will go. With L1 regularization (c1 parameter) coefficients of most features should be driven to zero. Let's check what effect does regularization have on CRF weights:

In [7]:
crf = sklearn_crfsuite.CRF(
    algorithm='lbfgs',
    c1=200,
    c2=0.1,
    max_iterations=20,
    all_possible_transitions=False,
)
crf.fit(X_train, y_train)
eli5.show_weights(crf, top=30)

As you can see, memoized tokens are mostly gone and model now relies on word shapes and POS tags. There is only a few non-zero features remaining. In our example the change probably made the quality worse, but that's a separate question.

Let's focus on transition weights. We can expect that O -> I-ENTIRY transitions to have large negative weights because they are impossible. But these transitions have zero weights, not negative weights, both in heavily regularized model and in our initial model. Something is going on here. 

The reason they are zero is that crfsuite haven't seen these transitions in training data, and assumed there is no need to learn weights for them, to save some computation time. This is the default behavior, but it is possible to turn it off using sklearn_crfsuite.CRF ``all_possible_transitions`` option. Let's check how does it affect the result:

In [8]:
crf = sklearn_crfsuite.CRF(
    algorithm='lbfgs',
    c1=0.1, 
    c2=0.1, 
    max_iterations=20, 
    all_possible_transitions=True,
)
crf.fit(X_train, y_train);

In [9]:
eli5.show_weights(crf, top=5, show=['transition_features'])

From \ To,O,B-LOC,I-LOC,B-MISC,I-MISC,B-ORG,I-ORG,B-PER,I-PER
O,2.732,1.217,-4.675,1.515,-5.785,1.36,-6.19,0.968,-6.236
B-LOC,-0.226,-0.091,3.378,-0.433,-1.065,-0.861,-1.783,-0.295,-1.57
I-LOC,-0.184,-0.585,2.404,-0.276,-0.485,-0.582,-0.749,-0.442,-0.647
B-MISC,-0.714,-0.353,-0.539,-0.278,3.512,-0.412,-1.047,-0.336,-0.895
I-MISC,-0.697,-0.846,-0.587,-0.297,4.252,-0.84,-1.206,-0.523,-1.001
B-ORG,0.419,-0.187,-1.074,-0.567,-1.607,-1.13,5.392,-0.223,-2.122
I-ORG,-0.117,-1.715,-0.863,-0.631,-1.221,-1.442,5.141,-0.397,-1.908
B-PER,-0.127,-0.806,-0.834,-0.52,-1.228,-1.089,-2.076,-1.01,4.04
I-PER,-0.766,-0.242,-0.67,-0.418,-0.856,-0.903,-1.472,-0.692,2.909


With `all_possible_transitions=True` CRF learned large negative weights for impossible transitions like O -> I-ORG.

## 5. Customization

The table above is large and kind of hard to inspect; eli5 provides several options to look only at a part of features. You can check only a subset of labels:

In [10]:
eli5.show_weights(crf, top=10, targets=['O', 'B-ORG', 'I-ORG'])

Another option is to check only some of the features - it helps to check if a feature function works as intended. For example, let's check how word shape features are used by model using ``feature_re`` argument and hide transition table:

In [11]:
eli5.show_weights(crf, top=10, feature_re='^word\.is', 
                  horizontal_layout=False, show=['targets'])

Weight?,Feature
-3.685,word.isupper()
-7.025,word.istitle()
Weight?,Feature
+2.397,word.istitle()
+0.099,word.isupper()
-0.152,word.isdigit()
Weight?,Feature
+0.460,word.istitle()
-0.018,word.isdigit()
-0.345,word.isupper()


Looks fine - UPPERCASE and Titlecase words are likely to be entities of some kind.

## 6. Formatting in console

It is also possible to format the result as text (could be useful in console):

In [12]:
expl = eli5.explain_weights(crf, top=5, targets=['O', 'B-LOC', 'I-LOC'])
print(eli5.format_as_text(expl))

Explained as: CRF

Transition features:
            O    B-LOC    I-LOC
-----  ------  -------  -------
O       2.732    1.217   -4.675
B-LOC  -0.226   -0.091    3.378
I-LOC  -0.184   -0.585    2.404

y='O' top features
Weight  Feature       
------  --------------
+4.931  BOS           
+3.754  postag[:2]:Fp 
+3.539  bias          
… 15043 more positive …
… 3906 more negative …
-3.685  word.isupper()
-7.025  word.istitle()

y='B-LOC' top features
Weight  Feature           
------  ------------------
+2.397  word.istitle()    
+2.147  -1:word.lower():en
  … 2284 more positive …  
  … 433 more negative …   
-1.080  postag[:2]:SP     
-1.080  postag:SP         
-1.273  -1:word.istitle() 

y='I-LOC' top features
Weight  Feature           
------  ------------------
+0.882  -1:word.lower():de
+0.780  -1:word.istitle() 
+0.718  word[-3:]:de      
+0.711  word.lower():de   
  … 1684 more positive …  
  … 268 more negative …   
-1.965  BOS               

